# Download e caricamento librerie

In [1]:
%tensorflow_version  2.x
! pip install -q keras==2.3.0

     |████████████████████████████████| 378kB 3.8MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 


In [2]:
!pip install ipython-autotime
%load_ext autotime

time: 158 µs


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 33.7 s


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13666207687353463035, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7256876049722361655
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 277200799048611111
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3388253617335944846
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

time: 7.08 s


In [5]:
!pip install contractions

     |████████████████████████████████| 317kB 5.9MB/s 
     |████████████████████████████████| 245kB 24.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81697 sha256=67d5d9fcf2e630d0513b2df96b16c160a893ff75a13850d7693c764146a80296
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
time: 6.46 s


In [6]:
!pip install lime

     |████████████████████████████████| 276kB 4.7MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283845 sha256=a7fb02f5ed81bc137cc801e5825e663399e86c7abe161cea8adc07384466cbe3
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime
time: 4.24 s


In [7]:
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import numpy as np
import matplotlib.pyplot as plt
import nltk
import regex as re
import seaborn as sb
from datetime import datetime
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import json
import functools
import itertools
import contractions
from collections import Counter
import ast

import sklearn
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier


from keras.models import Sequential
from keras import layers, callbacks, optimizers
import seaborn as sn


import scipy.stats

# per usare progress_apply
tqdm_notebook().pandas()

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download("punkt")

pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
time: 2.8 s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


# Load Dataset e prime analisi descrittive

In [8]:
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

time: 6.52 ms


In [9]:
%%capture
# cd_vinyl
!wget "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/CDs_and_Vinyl_5.json.gz"
df_1 = getDF("/content/CDs_and_Vinyl_5.json.gz")

# Food
!wget "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz"
df_2 = getDF("/content/Grocery_and_Gourmet_Food_5.json.gz")

# Cell Phone
!wget "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz"
df_3 = getDF("/content/Cell_Phones_and_Accessories_5.json.gz")

# Sport
!wget "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Sports_and_Outdoors_5.json.gz"
df_4 = getDF("/content/Sports_and_Outdoors_5.json.gz")

time: 3min 15s


In [10]:
df_1.loc[:2]

,reviewerID,asin,reviewerName,verified,reviewText,overall,reviewTime,summary,unixReviewTime,style,vote,image
0,A1H1DL4K669VQ9,0001393774,Judith Paladino,True,Love it!! Great seller!,5.0,"04 29, 2016",Five Stars,1461888000,NaN,NaN,NaN
1,A3V5XBBT7OZG5G,0001393774,gflady,True,One of my very favourite albums from one of my very favourite singers. I was happy to see I could replace the old worn cassettes from years ago.,5.0,"02 23, 2016",One of my very favourite albums from one of my very favourite singers,1456185600,NaN,NaN,NaN
2,A3SNL7UJY7GWBI,0001393774,Lady Leatherneck,True,"THank you Jesus Lord God, that brother Green's music is still sounding though he is Home with you now.",5.0,"02 11, 2016",Five Stars,1455148800,NaN,NaN,NaN


time: 50.4 ms


In [11]:
n_review_1 = df_1.groupby('asin')['reviewerID'].count().sort_values(ascending=False)

n_review = n_review_1

time: 680 ms


In [12]:
n_review_2 = df_2.groupby('asin')['reviewerID'].count().sort_values(ascending=False)

n_review = pd.concat([n_review,n_review_2])

time: 559 ms


In [13]:
n_review_3 = df_3.groupby('asin')['reviewerID'].count().sort_values(ascending=False)

n_review = pd.concat([n_review,n_review_3])

time: 579 ms


In [14]:
n_review_4 = df_4.groupby('asin')['reviewerID'].count().sort_values(ascending=False)

n_review = pd.concat([n_review,n_review_4])

time: 1.37 s


In [15]:
n_review.sort_values(ascending=False)

asin
B00BUKL666    7387
B008QMX2SG    6228
B00D3M2QP4    6221
B0010O748Q    4163
B00R7PWK7W    3387
              ... 
B0013FBFFU    1   
B0012IZ4JG    1   
B0013LKXEI    1   
B0012FY7TW    1   
B000ZKAY8I    1   
Name: reviewerID, Length: 267906, dtype: int64

time: 147 ms


In [16]:
datetime.utcfromtimestamp(df_1.loc[100]['unixReviewTime']).strftime('%Y-%m-%d %H:%M:%S')

'2013-02-22 00:00:00'

time: 3.76 ms


In [17]:
# Percentuale missing values
print("Missing df_1")
print(df_1.isna().sum()/len(df_1))
print("Missing df_2")
print(df_2.isna().sum()/len(df_2))
print("Missing df_3")
print(df_3.isna().sum()/len(df_3))
print("Missing df_4")
print(df_4.isna().sum()/len(df_4))

Missing df_1
reviewerID        0.000000
asin              0.000000
reviewerName      0.000037
verified          0.000000
reviewText        0.000194
overall           0.000000
reviewTime        0.000000
summary           0.000157
unixReviewTime    0.000000
style             0.038196
vote              0.660529
image             0.997428
dtype: float64
Missing df_2
overall           0.000000
verified          0.000000
reviewTime        0.000000
reviewerID        0.000000
asin              0.000000
reviewerName      0.000121
reviewText        0.000341
summary           0.000191
unixReviewTime    0.000000
vote              0.861695
style             0.482379
image             0.991686
dtype: float64
Missing df_3
overall           0.000000
verified          0.000000
reviewTime        0.000000
reviewerID        0.000000
asin              0.000000
style             0.463647
reviewerName      0.000120
reviewText        0.000678
summary           0.000458
unixReviewTime    0.000000
vote         

In [18]:
# Eliminazione elementi senza recensioni
df_1.dropna(subset=["reviewText"], inplace = True)
df_2.dropna(subset=["reviewText"], inplace = True)
df_3.dropna(subset=["reviewText"], inplace = True)
df_4.dropna(subset=["reviewText"], inplace = True)

time: 3.86 s


In [19]:
# Hanno come separatore delle migliaia la "," che va eliminata
df_1.vote = df_1.vote.str.replace(",", "")
df_1.vote = df_1.vote.astype("float")

df_2.vote = df_2.vote.str.replace(",", "")
df_2.vote = df_2.vote.astype("float")

df_3.vote = df_3.vote.str.replace(",", "")
df_3.vote = df_3.vote.astype("float")

df_4.vote = df_4.vote.str.replace(",", "")
df_4.vote = df_4.vote.astype("float")

time: 2.66 s


## Rimozione URL e aggiunta lunghezza per ogni recensione e variabile target

In [20]:
# Rimozione url 
def remove_url(x):
  x =  re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", 
                '', x) # rimozione url
  x = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', ' ', x) # rimozione tag HTML
  return x

df_1['reviewText'] = df_1['reviewText'].progress_apply(lambda x: remove_url(x))
df_2['reviewText'] = df_2['reviewText'].progress_apply(lambda x: remove_url(x))
df_3['reviewText'] = df_3['reviewText'].progress_apply(lambda x: remove_url(x))
df_4['reviewText'] = df_4['reviewText'].progress_apply(lambda x: remove_url(x))



time: 7min 33s


In [21]:
# Aggiunta variabile per lunghezza review
df_1["review_length"] = df_1["reviewText"].progress_apply(lambda x: len(x))
df_2["review_length"] = df_2["reviewText"].progress_apply(lambda x: len(x))
df_3["review_length"] = df_3["reviewText"].progress_apply(lambda x: len(x))
df_4["review_length"] = df_4["reviewText"].progress_apply(lambda x: len(x))


time: 13.2 s


In [22]:
df_1 = df_1.sort_values('review_length', ascending = False).groupby('asin').first()
df_2 = df_2.sort_values('review_length', ascending = False).groupby('asin').first()
df_3 = df_3.sort_values('review_length', ascending = False).groupby('asin').first()
df_4 = df_4.sort_values('review_length', ascending = False).groupby('asin').first()

time: 43.6 s


In [23]:
len(df_1) + len(df_2) + len(df_3) + len(df_4)

267906

time: 3.46 ms


In [24]:
# Aggiunta variabile target

df_1['target'] = 'CD'
df_2['target'] = 'Food'
df_3['target'] = 'Cell'
df_4['target'] = 'Sport'

time: 8.1 ms


In [25]:
# Concatenazione dei vari dataframe
df = pd.concat([df_1[0:10261], df_2[0:10261], df_3[0:10261], df_4[0:10261]])

time: 86.8 ms


In [26]:
len(df)

41044

time: 2.87 ms


In [27]:
df.reset_index()
df.head()

reviewerID          reviewerName  ...  review_length target
asin                                              ...                      
0001393774  AEKGGV851HY3K   Avid Reader           ...  564            CD   
0005164885  A9VROMAWXIL9G   Roman Midnight Music  ...  5259           CD   
0005465079  A14YR7QK3ASFFW  Amazon Customer       ...  367            CD   
0739040251  A2PI3IIBPG2VI2  Phil Sac              ...  1642           CD   
0788602144  A2CS77CAOHQV02  Don E. Marchione      ...  1542           CD   

[5 rows x 13 columns]

time: 48.4 ms


In [28]:
#df.to_csv('/content/drive/My Drive/Text Mining Progetto/Classification/df_classification.csv')

time: 847 µs


# Preprocessing

In [29]:
#data_clean = pd.read_csv('/content/drive/My Drive/Text Mining Progetto/Classification/df_classification.csv')

time: 513 µs


In [30]:
data_clean = df.copy()

time: 12.4 ms


In [31]:
data_clean.dtypes

reviewerID        object 
reviewerName      object 
verified          bool   
reviewText        object 
overall           float64
reviewTime        object 
summary           object 
unixReviewTime    int64  
style             object 
vote              float64
image             object 
review_length     int64  
target            object 
dtype: object

time: 5.06 ms


In [32]:
data_clean.drop(columns=['image','style','unixReviewTime','reviewTime',
                         'verified','overall','reviewerID','reviewerName'],
                inplace=True)

time: 34.5 ms


## Tokenization

In [33]:
def tokenize_only(text):
  text = text.replace('/', " ")
  text = text.replace('=', " ")
  text = text.replace('_', " ")
  text = text.replace('-', " ")
  text = text.replace('.', " ")
  text = text.replace('+', " ")
  text = text.replace('\\', " ")
  # tokenization e case folding
  tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
  filtered_tokens = []
  for token in tokens:
    if re.search('[a-zA-Z]', token):
      token = re.sub(r'[^\w]', '', token) # rimozione simboli
      token = re.sub(r'[0-9]+', '', token) # rimozione numeri
      filtered_tokens.append(token)
  return filtered_tokens

time: 9.1 ms


In [34]:
data_clean['reviewText_token'] = data_clean['reviewText'].progress_apply(lambda x: tokenize_only(x))


time: 4min 6s


In [35]:
data_clean.head()

reviewText  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            reviewText_token
asin                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

time: 32 ms


In [36]:
# salvataggio data_clean
#data_clean.to_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_token.csv", index = False)

time: 859 µs


In [37]:
#data_clean = pd.read_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_token.csv")
#data_clean['reviewText_token'] = data_clean['reviewText_token'].progress_apply(lambda x: ast.literal_eval(x))

time: 650 µs


## Normalization

In [38]:
def manage_contractions_slang(el):
  ris=contractions.fix(el,slang=True)
  return ris

time: 1.21 ms


In [39]:
data_clean['reviewText_token'] = data_clean.reviewText_token.progress_apply(lambda x: list(map(manage_contractions_slang, x)))


time: 34.8 s


In [40]:
data_clean.head()

reviewText  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            reviewText_token
asin                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

time: 35.8 ms


In [41]:
# salvataggio data_prep
#data_clean.to_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_norm.csv", index = False)

time: 970 µs


In [42]:
#data_clean = pd.read_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_norm.csv")
#data_clean['reviewText_token'] = data_clean['reviewText_token'].progress_apply(lambda x: ast.literal_eval(x))

time: 954 µs


## Stemming

In [43]:
data_clean.head()

reviewText  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            reviewText_token
asin                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

time: 31.4 ms


In [44]:
def create_text(x):
  'data una lista di parole'
  ' torna il testo'
  res = ' '.join(x)
  return res

time: 2.15 ms


In [45]:
data_clean['text'] = data_clean['reviewText_token'].progress_apply(lambda x: create_text(x))


time: 528 ms


In [46]:
from nltk.stem import SnowballStemmer
ss = SnowballStemmer("english")
def getStemming(string):
  output = []
  for word in string.split(" "):
    output.append(ss.stem(word))
  return output

data_clean["text_stemmed"] = data_clean.progress_apply(lambda row:getStemming(row["text"]),axis=1)


time: 2min 7s


In [47]:
data_clean.head()

reviewText  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           text_stemmed
asin                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

time: 46.7 ms


In [48]:
# salvataggio data_clean
#data_clean.to_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_stem.csv", index = False)

time: 1 ms


In [49]:
#data_clean = pd.read_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_stem.csv")
#data_clean['reviewText_token'] = data_clean['reviewText_token'].progress_apply(lambda x: ast.literal_eval(x))
#data_clean['text_stemmed'] = data_clean['text_stemmed'].progress_apply(lambda x: ast.literal_eval(x))

time: 1.24 ms


## Stop words

In [50]:
data_clean.head()

reviewText  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           text_stemmed
asin                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

time: 41.3 ms


In [51]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['could','would','might','never','often','usually', 'nt',
                  'sometimes','always','just','already','yet','still',
                  'product','item','price','amazon', 'prime', 'buy', 'purchase',
                  'shipping','gift','seller', 'new','customer','sell', 'deal',
                  'order','brand','receive'])

time: 5.6 ms


In [52]:
data_clean.iloc[8]

reviewText          Okay, so it's a 17 year old concert & the only official one available of La Benatar on the market.  However, what this TV Special showcases is the magnitude of Benatar's sizzlingly raucous & sweet voice as she strums through megahits up to her Get Nervous multi-platinum album, including grammy-award winning Fire & Ice, as well as Hit Me With Your Best Shot & Heartbreaker.  What is most surprising about this performance, particularly for lesser fans, is that the album trax are the more astounding performances, both tune-wise and vocally.  The title track to Pat's first album In The Heat of The Night is a blindingly electric and sultry hot in the city type trak and the wondrous single Shadows of The Night sends shivers up your back from the opening guitar chords.  Personal faves are the medley I Want Out & We Live For Love, which really wakens the crowd up & a rip -roaring rendition of You Better Run, where Pat brutally warns off an offending lover.  The eloquent rock

time: 4.32 ms


In [53]:
data_clean['text_stemmed'] = data_clean['text_stemmed'].progress_apply(lambda x: list(filter(lambda y: y not in stopwords, x)))


time: 22.1 s


In [54]:
# Rimozione eventuali null
data_clean['text_stemmed'].dropna(inplace = True)

time: 6.35 ms


In [55]:
data_clean.iloc[8]

reviewText          Okay, so it's a 17 year old concert & the only official one available of La Benatar on the market.  However, what this TV Special showcases is the magnitude of Benatar's sizzlingly raucous & sweet voice as she strums through megahits up to her Get Nervous multi-platinum album, including grammy-award winning Fire & Ice, as well as Hit Me With Your Best Shot & Heartbreaker.  What is most surprising about this performance, particularly for lesser fans, is that the album trax are the more astounding performances, both tune-wise and vocally.  The title track to Pat's first album In The Heat of The Night is a blindingly electric and sultry hot in the city type trak and the wondrous single Shadows of The Night sends shivers up your back from the opening guitar chords.  Personal faves are the medley I Want Out & We Live For Love, which really wakens the crowd up & a rip -roaring rendition of You Better Run, where Pat brutally warns off an offending lover.  The eloquent rock

time: 6.05 ms


In [56]:
data_clean['text'] = data_clean['text_stemmed'].progress_apply(lambda x: create_text(x))


time: 368 ms


In [57]:
# salvataggio data_clean
# data_clean.to_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_stop.csv", index = False)

time: 1.12 ms


# Rimozione parole molto/poco frequenti (non effettuata perchè non necessario)

In [ ]:
#data_clean = pd.read_csv("/content/drive/My Drive/Text Mining Progetto/Classifiction/data_prep.csv")
#data_clean['reviewText_token'] = data_clean['reviewText_token'].progress_apply(lambda x: ast.literal_eval(x))
#data_clean['text_lemma'] = data_clean['text_lemma'].progress_apply(lambda x: ast.literal_eval(x))

In [ ]:
cnt = Counter()

for text in data_clean["text"].values:
  for word in str(text).split():
    cnt[word] += 1

In [ ]:
rare_words = 10
rarewords = set([w for (w, wc) in cnt.most_common()[:-rare_words-1:-1]])

In [ ]:
rarewords

In [ ]:
df = pd.DataFrame.from_dict(cnt, orient='index').reset_index()
df.rename(columns={'index': 'word', 0:'count'}, inplace = True)

In [ ]:
df["length"] = df["word"].apply(lambda x: len(x))

In [ ]:
df.sort_values(by="length", ascending = False).head(20)

In [ ]:
# controllo commenti con una parola
df_3['check'] = df_3['reviewText'].apply(lambda x: 'Aveeno' in x)
df_3[df_3['check'] == True]

In [ ]:
# controllo commenti con una parola
data_clean['check'] = data_clean['text'].apply(lambda x: 'ooeeooahahtwingtwangwallawallabingbang' in x)
data_clean[data_clean['check'] == True]

### Rimozione parole frequenti

In [ ]:
# rimozione parole presenti una volta
df_sub = df[df['count'] > 1].copy()

In [ ]:
plt.boxplot(df_sub['count'])

In [ ]:
plt.hist(df_sub['count'])

In [ ]:
df.sort_values(by = 'count', ascending = False, inplace = True)
df.head(20)

### Rimozione parole poco frequenti

In [ ]:
# rimozione parole presenti una volta
df_sub = df[(df['count'] > 1) & (df['count'] <25000) & (df['length'] > 1)].copy()
#df_sub = df[(df['count'] > 7) & (df['count'] <500) & (df['length'] > 1)].copy()

In [ ]:
plt.boxplot(df_sub['count'])

In [ ]:
# rimozione parole con lunghezza 1
word_to_remove = df[df['length'] == 1]['word']
# rimozione parole > 25000
word_to_remove = word_to_remove.append(df[df['count'] > 25000]['word'])
# rimozione parole < 7 (mediana)
word_to_remove = word_to_remove.append(df[df['count'] <= 7]['word'])

In [ ]:
word_to_remove = set(word_to_remove)

In [ ]:
len(word_to_remove)

In [ ]:
def remove_words(text):
  return " ".join([word for word in str(text).split() if word not in word_to_remove])

data = data_prep.copy()
data["sentence_lemma"] = data["sentence_lemma"].progress_apply(lambda text: remove_words(text))

In [ ]:
print(len(word_to_remove))
print(len(df))
print(round(len(word_to_remove)/len(df) * 100, 2))

In [ ]:
data.to_csv("/content/drive/My Drive/Text Mining Progetto/data.csv", index = False)

# Rappresentazione TF-IDF

In [58]:
#data = pd.read_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_stop.csv")
#data['reviewText_token'] = data['reviewText_token'].progress_apply(lambda x: ast.literal_eval(x))
#data['text_stemmed'] = data['text_stemmed'].progress_apply(lambda x: ast.literal_eval(x))
data = data_clean.copy()

time: 9.79 ms


In [59]:
data.dropna(subset=["text"], inplace = True)

#shuffle dati
data = data.sample(frac=1)

time: 69.6 ms


In [61]:
data.head(1)

,reviewText,summary,vote,review_length,target,reviewText_token,text,text_stemmed
asin,,,,,,,,
B000002GPD,"Alone is the first song to be on the top 30 in US since the album Spirits Having Flown. This song is not just led by one lead vocal, but two, who were Robin Gibb, and Barry Gibb. With Maurice and Barry on guitar, Steve on drum, Alan on rhythem guitar, Simon on keyboards and a bass guitar, the song sounded better than any other songs.\n Stayin' Alive was written before Saturday Night Fever was even out, so you can see how good the Bee Gees can rock. The song in this CD is live, so good to hear how Barry Gibb uses his falsetto voice in the live concert.\n The other two songs are also excellent. You Should Be Dancing is Bee Gees' first super-rock song to be US Chart #1. The other song never had been in any other albums, so get a copy and listen to it!\n My last word is - A GREAT SINGLE.",The Rocking Song,3.0,794,CD,"[alone, is, the, first, song, to, be, on, the, top, in, us, since, the, album, spirits, having, flown, this, song, is, not, just, led, by, one, lead, vocal, but, two, who, were, robin, gibb, and, barry, gibb, with, maurice, and, barry, on, guitar, steve, on, drum, alan, on, rhythem, guitar, simon, on, keyboards, and, a, bass, guitar, the, song, sounded, better, than, any, other, songs, stayin, alive, was, written, before, saturday, night, fever, was, even, out, so, you, can, see, how, good, the, bee, gees, can, rock, the, song, in, this, cd, is, live, so, good, to, hear, how, barry, ...]",alon first song top us sinc album spirit flown song led one lead vocal two robin gibb barri gibb mauric barri guitar steve drum alan rhythem guitar simon keyboard bass guitar song sound better ani song stayin aliv written befor saturday night fever even see good bee gee rock song cd live good hear barri gibb use falsetto voic live concert two song also excel danc bee gee first super rock song us chart song ani album get copi listen last word great singl,"[alon, first, song, top, us, sinc, album, spirit, flown, song, led, one, lead, vocal, two, robin, gibb, barri, gibb, mauric, barri, guitar, steve, drum, alan, rhythem, guitar, simon, keyboard, bass, guitar, song, sound, better, ani, song, stayin, aliv, written, befor, saturday, night, fever, even, see, good, bee, gee, rock, song, cd, live, good, hear, barri, gibb, use, falsetto, voic, live, concert, two, song, also, excel, danc, bee, gee, first, super, rock, song, us, chart, song, ani, album, get, copi, listen, last, word, great, singl]"


time: 25.5 ms


In [62]:
corpus = data["text"]
target = data["target"]

time: 1.18 ms


In [63]:
def crea_lista(element):
  #lista = list(map(ottieni_elemento, element))
  lista = element.split(' ')
  return lista

time: 1.53 ms


In [65]:
lista_parole = list(map(crea_lista, corpus))
dizionario = list(itertools.chain(*lista_parole))
len(dizionario)

5886962

time: 1.17 s


In [66]:
len(Counter(dizionario))

72102

time: 1.06 s


In [67]:
tfidf_vectorizer = TfidfVectorizer(max_df = 0.8,
                                 #max_features = 200000,
                                 min_df = 10, 
                                 ngram_range = (1,1))

time: 1.45 ms


In [68]:
tf_idf_matrix = tfidf_vectorizer.fit_transform(corpus)

time: 4.72 s


In [69]:
print(tf_idf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()

(41044, 15005)
time: 16.7 ms


In [70]:
from numpy import count_nonzero
1.0 - (count_nonzero(tf_idf_matrix.toarray()) / float(tf_idf_matrix.toarray().size))

0.9937009643116395

time: 5.99 s


## Classificazione

In [ ]:
# Per intervalli di confidenza

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
# Risultati cross validation 

def cross_score(model, X, y, k_fold):
    cval_score = []
    i = 1
    Kfold = StratifiedKFold(k_fold, shuffle=True, random_state=42)
    for train_index, test_index in Kfold.split(X, y):
        
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y.values[list(train_index)], y.values[list(test_index)]

        if str(type(model)) == "<class 'keras.engine.sequential.Sequential'>":
            
            label_encoder = LabelEncoder()
            y_train = label_encoder.fit_transform(y_train)
            y_val = label_encoder.fit_transform(y_val)
            model.fit(X_train, y_train,
                      verbose=0,
                      epochs=10,
                      validation_split=0.2,
                      batch_size=64,
                      callbacks=[es])
            y_pred = model.predict_classes(X_val)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
        
        actual = sklearn.metrics.accuracy_score(y_val, y_pred)
        cval_score.append(actual)
        print("Numero Fold: " + str(i))
        i = i+1

    result = mean_confidence_interval(cval_score)   
    return result

### SVM

In [ ]:
svm_classifier = svm.LinearSVC(C = 1)

In [ ]:
cross_score(svm_classifier, tf_idf_matrix, target, 10)

### Random Forest

In [ ]:
cross_score(rf, tf_idf_matrix, target, 10)

### Neural Network

In [ ]:
es = callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [ ]:
# Creazione modello
input_dim = tf_idf_matrix.shape[1]

nn = Sequential()
nn.add(layers.Dense(64, input_dim=input_dim, activation='relu'))
nn.add(layers.Dropout(0.3))
nn.add(layers.Dense(4, activation='softmax'))

# Compilazione modello
nn.compile(loss='sparse_categorical_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
nn.summary()

In [ ]:
cross_score(nn, tf_idf_matrix, target, 10)

#### Explainable AI

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(tf_idf_matrix, target, test_size = 0.2, random_state = 1521)

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [ ]:
input_dim = tf_idf_matrix.shape[1]

# Creazione modello
nn = Sequential()
nn.add(layers.Dense(64, input_dim=input_dim, activation='relu'))
nn.add(layers.Dropout(0.3))
nn.add(layers.Dense(4, activation='softmax'))

# Compilazione modello
nn.compile(loss='sparse_categorical_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])

In [ ]:
history = nn.fit(x_train, y_train,
                    epochs=10,
                    validation_split=0.2,
                    batch_size=64,
                    callbacks = [es])

In [ ]:
nn.evaluate(x_test, y_test)[1]

In [ ]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(tfidf_vectorizer, nn)

In [ ]:
from lime.lime_text import LimeTextExplainer

class_names = ['CD', 'Cell', 'Food', 'Sport']
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
def explain_text(idx):
  print('Document id: %d' % idx)
  print('Predicted class =', class_names[np.argmax(c.predict_proba([corpus[idx]])[0])])
  print('True class: %s' % target[idx])
  print()
  exp = explainer.explain_instance(corpus[idx], c.predict_proba, num_features=10, top_labels=4)
  exp.show_in_notebook(text=True)

In [ ]:
explain_text(60002)

# Word Embedding

In [ ]:
#data = pd.read_csv("/content/drive/My Drive/Text Mining Progetto/Classification/data_clean_stop.csv")
#data['reviewText_token'] = data['reviewText_token'].progress_apply(lambda x: ast.literal_eval(x))
#data['text_stemmed'] = data['text_stemmed'].progress_apply(lambda x: ast.literal_eval(x))

data = data_clan.copy()

In [ ]:
data.dropna(subset=["text"], inplace = True)

#shuffle dati
data = data.sample(frac=1)

In [ ]:
data.head(1)

In [ ]:
corpus_wd = data['text_stemmed']

## Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
document_tagged = data.progress_apply(lambda x: TaggedDocument(words=x['text_stemmed'],
                                      tags=[x['target']]), axis=1)

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=30, hs=0, min_count=6)
model_dbow.build_vocab([x for x in tqdm_notebook(document_tagged.values)])

In [ ]:
# Modello Distributed bag of words

from sklearn import utils
for epoch in tqdm_notebook(range(10), desc= 'Training model'):
    model_dbow.train(utils.shuffle([x for x in document_tagged.values]), total_examples=len(document_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [ ]:
def labeled_vectors(model:Doc2Vec,
                     tagged_docs:pd.core.series.Series):
  sents = tagged_docs.values
  targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps = 10)) for doc in tqdm_notebook(sents)])
  return targets, regressors

In [ ]:
y, x = labeled_vectors(model_dbow, document_tagged)

### Classificazione

#### SVM

In [ ]:
svm_classifier = svm.LinearSVC(C=1)

In [ ]:
cross_score(svm_classifier, np.asarray(x), pd.Series(y), 10)

#### Random Forest

In [ ]:
cross_score(rf, np.asarray(x), pd.Series(y), 10)

#### Neural Network

In [ ]:
# Creazione modello
input_dim = np.asarray(x).shape[1]

nn = Sequential()
nn.add(layers.Dense(64, input_dim=input_dim, activation='relu'))
nn.add(layers.Dropout(0.3))
nn.add(layers.Dense(4, activation='softmax'))

# Compilazione modello
nn.compile(loss='sparse_categorical_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
nn.summary()

In [ ]:
cross_score(nn, np.asarray(x), pd.Series(y), 10)